# Scikit-learn + Neptune

Neptune helps you keep track of your machine learning runs and if you are using scikit-learn you can add tracking very easily.

This quickstart will show you how to (using just single function) log scikit-learn:

* regression summary,
* classification summary,
* kmeans clustering summary.

## Before you start

### Install dependencies

In [ ]:
! pip install scikit-learn==0.24.1 neptune-client==0.5.5 neptune-sklearn==0.9.1

In [ ]:
! pip install --upgrade scikit-learn neptune-client neptune-sklearn

## Scikit-learn regression

### Step 1: Create and fit random forest regressor

Define regressor parameters, that will be later passed to Neptune.

In [ ]:
parameters = {'n_estimators': 70,
              'max_depth': 7,
              'min_samples_split': 3}

Create and fit regressor.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

rfr = RandomForestRegressor(**parameters)

X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=28743)

rfr.fit(X_train, y_train)

### Step 2: Initialize Neptune

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project='common/sklearn-integration',
                   api_token='ANONYMOUS',
                   name='regression-example',
                   tags=['RandomForestRegressor', 'regression'])

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next.

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `'common/sklearn-integration'` as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project_qualified_name='my_workspace/my_project', 
             api_token='MY_API_TOKEN')
```

### Step 3: Log regressor summary

In [ ]:
import neptune.new.integrations.sklearn as npt_utils

run['rfr_summary'] = npt_utils.create_regressor_summary(rfr, X_train, X_test, y_train, y_test)

You just logged information about the regressor, including:

* [logged regressor parameters](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Fall_params),
* [logged pickled model](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2F&attribute=pickled_model),
* [logged test predictions](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Ftest&attribute=preds),
* [logged test scores](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Ftest%2Fscores),
* [logged regressor visualizations](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Fdiagnostics_charts&attribute=feature_importance),
* [logged metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/source-code?file=main.py).

In [ ]:
# tests
run.wait()

# check tags
all_tags = ['RandomForestRegressor', 'regression']
assert set(run["sys/tags"].fetch()) == set(all_tags), 'Expected: {}, Actual: {}'.format(all_tags, run["sys/tags"].fetch())

# check scores
assert run['rfr_summary/test/scores/explained_variance_score'].fetch() <= 1.0, 'Wrong values logged.'
assert run['rfr_summary/test/scores/max_error'].fetch() >= 0.0, 'Wrong values logged.'
assert run['rfr_summary/test/scores/mean_absolute_error'].fetch() >= 0.0, 'Wrong values logged.'
assert run['rfr_summary/test/scores/r2_score'].fetch() <= 1.0, 'Wrong values logged.'

# check parameters
for key in parameters.keys():
    assert run['rfr_summary/all_params/{}'.format(key)].fetch() == parameters[key],\
        'Expected: {}, Actual: {}'.format(parameters[key], run['rfr_summary/all_params/{}'.format(key)].fetch())

### Step 4: Explore results

You just learned how to log scikit-learn regression summary to Neptune using single function.

Click on the link that was outputted to the console or [go here](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all) to explore a run similar to yours. In particular check:

* [logged regressor parameters](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Fall_params),
* [logged pickled model](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2F&attribute=pickled_model),
* [logged test predictions](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Ftest&attribute=preds),
* [logged test scores](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Ftest%2Fscores),
* [logged regressor visualizations](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=rfr_summary%2Fdiagnostics_charts&attribute=feature_importance),
* [logged metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-92/source-code?file=main.py).

## Scikit-learn classification

### Step 1: Create and fit gradient boosting classifier

Define classifier parameters, that will be later passed to Neptune.

In [ ]:
parameters = {'n_estimators': 120,
              'learning_rate': 0.12,
              'min_samples_split': 3,
              'min_samples_leaf': 2}

Create and fit regressor.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

gbc = GradientBoostingClassifier(**parameters)

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=28743)

gbc.fit(X_train, y_train)

### Step 2: Initialize Neptune

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project='common/sklearn-integration',
                   api_token='ANONYMOUS',
                   name='classification-example',
                   tags=['GradientBoostingClassifier', 'classification'])

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next.

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `'common/sklearn-integration'` as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project_qualified_name='my_workspace/my_project', 
             api_token='MY_API_TOKEN')
```

### Step 3: Log classifier summary

In [ ]:
import neptune.new.integrations.sklearn as npt_utils

run['cls_summary'] = npt_utils.create_classifier_summary(gbc, X_train, X_test, y_train, y_test)

You just logged information about the classifier, including:

* [logged classifier parameters](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Fall_params),
* [logged pickled model](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2F&attribute=pickled_model),
* [logged test predictions](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Ftest&attribute=preds),
* [logged test predictions probabilities](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Ftest&attribute=preds_proba),
* [logged test scores](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Ftest%2Fscores%2F),
* [logged classifier visualizations](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Fdiagnostics_charts&attribute=class_prediction_error),
* [logged metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/source-code?file=main.py&filePath=integrations%2Fsklearn%2F).

In [ ]:
# tests
run.wait()

# check tags
all_tags = ['GradientBoostingClassifier', 'classification']
assert set(run["sys/tags"].fetch()) == set(all_tags), 'Expected: {}, Actual: {}'.format(all_tags, run["sys/tags"].fetch())

# check parameters
for key in parameters.keys():
    assert run['cls_summary/all_params/{}'.format(key)].fetch() == parameters[key],\
        'Expected: {}, Actual: {}'.format(parameters[key], run['cls_summary/all_params/{}'.format(key)].fetch())

### Step 4: Explore Results

You just learned how to log scikit-learn classification summary to Neptune using single function.

Click on the link that was outputted to the console or [go here](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/allhttps://ui.neptune.ai/o/shared/org/sklearn-integration/e/SKLEARN-312/charts) to explore a run similar to yours. In particular check:

* [logged classifier parameters](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Fall_params),
* [logged pickled model](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2F&attribute=pickled_model),
* [logged test predictions](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Ftest&attribute=preds),
* [logged test predictions probabilities](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Ftest&attribute=preds_proba),
* [logged test scores](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Ftest%2Fscores%2F),
* [logged classifier visualizations](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=cls_summary%2Fdiagnostics_charts&attribute=class_prediction_error),
* [logged metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-95/source-code?file=main.py&filePath=integrations%2Fsklearn%2F).

## Scikit-learn KMeans clustering

### Step 1: Create KMeans object and example data

Define KMeans clustering parameters, that will be later passed to Neptune.

In [ ]:
parameters = {'n_init': 11,
              'max_iter': 270}

Create and fit KMeans model.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

km = KMeans(**parameters)

X, y = make_blobs(n_samples=579, n_features=17, centers=7, random_state=28743)

### Step 2: Initialize Neptune

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project='common/sklearn-integration',
                   api_token='ANONYMOUS',
                   name='clustering-example',
                   tags=['KMeans', 'clustering'])

Click on the link above to open this run in Neptune. For now it is empty but keep the tab with run open to see what happens next.

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project `'common/sklearn-integration'` as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of the ``neptune.init()``.

For example:

```python
neptune.init(project_qualified_name='my_workspace/my_project', 
             api_token='MY_API_TOKEN')
```

### Step 3: Log KMeans clustering summary

In [ ]:
import neptune.new.integrations.sklearn as npt_utils

run['kmeans_summary'] = npt_utils.create_kmeans_summary(km, X, n_clusters=17)

You just logged information about the KMeans clustering, including:

* [logged KMeans parameters](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=kmeans_summary%2Fall_params),
* [logged cluster labels](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=kmeans_summary%2F&attribute=cluster_labels),
* [logged KMeans clustering visualizations](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=kmeans_summary%2Fdiagnostics_charts&attribute=silhouette),
* [logged metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/source-code?file=main.py&filePath=integrations%2Fsklearn%2F).

In [ ]:
# tests
run.wait()

# check tags
all_tags = ['KMeans', 'clustering']
assert set(run["sys/tags"].fetch()) == set(all_tags), 'Expected: {}, Actual: {}'.format(all_tags, run["sys/tags"].fetch())

# check parameters
for key in parameters.keys():
    assert run['kmeans_summary/all_params/{}'.format(key)].fetch() == parameters[key],\
        'Expected: {}, Actual: {}'.format(parameters[key], run['kmeans_summary/all_params/{}'.format(key)].fetch())

### Explore Results

You just learned how to log scikit-learn KMeans clustering summary to Neptune using single function.

Click on the link that was outputted to the console or [go here](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all) to explore a run similar to yours. In particular check:

* [logged KMeans parameters](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=kmeans_summary%2Fall_params),
* [logged cluster labels](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=kmeans_summary%2F&attribute=cluster_labels),
* [logged KMeans clustering visualizations](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=kmeans_summary%2Fdiagnostics_charts&attribute=silhouette),
* [logged metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/all?path=sys),
* [logged code and git metadata](https://app.neptune.ai/o/common/org/sklearn-integration/e/SKLEAR-96/source-code?file=main.py&filePath=integrations%2Fsklearn%2F).

## Other logging options

Neptune-Sklearn integration also lets you log only specific metadata of your choice, by using additional methods.

Below are few examples, visit [sklean integration documentation](https://docs-beta.neptune.ai/essentials/integrations/machine-learning-frameworks/sklearn) for the full example.

### Before you start: create and fit gradient boosting classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=28743)

rfc.fit(X_train, y_train)

### Import sklearn integration that will be used below, and create new run

In [ ]:
import neptune.new.integrations.sklearn as npt_utils

run = neptune.init(project='common/sklearn-integration',
                   api_token='ANONYMOUS',
                   name='other-options')

Open link above to see the metadata logging results, as we add them below.

### Log estimator parameters

In [ ]:
run['estimator/parameters'] = npt_utils.get_estimator_params(rfc)

### Log model

In [ ]:
run['estimator/pickled-model'] = npt_utils.get_pickled_model(rfc)

### Log confusion matrix

In [ ]:
run['confusion-matrix'] = npt_utils.create_confusion_matrix_chart(rfc, X_train, X_test, y_train, y_test)